<a href="https://colab.research.google.com/github/AABrom/Kaggle_nn1/blob/main/NLP2985dcf3db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

teta_nn_1_2025_path = kagglehub.competition_download('teta-nn-1-2025')

print('Data source import complete.')


100%|██████████| 20.3M/20.3M [00:02<00:00, 10.4MB/s]

Extracting files...


Data source import complete.


In [5]:
! pip install razdel pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 115.5 MB/s eta 0:00:00


In [6]:
# --- Системные и общие ---
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

# --- SKlearn ---
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# --- NLP-инструменты ---
import nltk
from nltk.corpus import stopwords
from razdel import tokenize as razdel_tokenize
import pymorphy3
from wordcloud import WordCloud

# --- PyTorch & Transformers ---
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import Dataset as HFDataset

# --- Визуализация ---
from graphviz import Digraph

# --- Константы и настройка среды ---
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Определяем устройство
if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.manual_seed(RANDOM_STATE)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(RANDOM_STATE)
else:
    device = torch.device("cpu")

print(f"✅ Используемое устройство: {device}")

# --- Инициализация лидерборда ---
leaderboard = pd.DataFrame(columns=['Метод', 'R2_Score'])

✅ Используемое устройство: cuda


In [7]:
full_train_df = pd.read_csv(teta_nn_1_2025_path +'/train.csv')
test_df = pd.read_csv(teta_nn_1_2025_path +'/test.csv')

# Разбиваем исходный train на обучающую и валидационную выборки
train_df, val_df = train_test_split(
    full_train_df,
    test_size=0.25,
    random_state=RANDOM_STATE
)

# Сбрасываем индексы для удобства
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

# Создаем единое текстовое поле для анализа
text_cols = ['title', 'location', 'company', 'skills', 'description']
for df in [train_df, val_df, test_df]:
    # Заполняем пропуски пустыми строками, чтобы избежать ошибок
    df[text_cols] = df[text_cols].fillna('')
    # Объединяем текстовые колонки через разделитель
    df['full_text'] = df[text_cols].agg(' | '.join, axis=1)

print("Пример данных из train:")
display(train_df[['full_text', 'log_salary_from']].head(3))

Пример данных из train:


,full_text,log_salary_from
0,QA Engineer | Москва (м. Сколково / м. Крылатс...,5.298317
1,Senior Backend QA engineer | Санкт-Петербург |...,5.010635
2,Python разработчик (Django) | Москва | Hammer ...,3.912023


In [8]:
# Загружаем стоп-слова
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('russian'))
# Добавим специфичные для вакансий стоп-слова
stop_words.update(['наш', 'компания', 'команда', 'работа', 'вакансия', 'искать'])

# Инициализируем лемматизатор pymorphy3
morph = pymorphy3.MorphAnalyzer()

def preprocess_text(text: str) -> str:
    """Полный цикл предобработки текста с использованием pymorphy3."""
    # 1. Токенизация
    tokens = [token.text for token in razdel_tokenize(text)]
    # 2. Приведение к нижнему регистру и удаление не-буквенных токенов
    lower_tokens = [token.lower() for token in tokens if token.isalpha()]
    # 3. Лемматизация и удаление стоп-слов
    lemmas = [morph.parse(token)[0].normal_form for token in lower_tokens]
    cleaned_lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
    return " ".join(cleaned_lemmas)

# Демонстрируем работу на одном примере
sample_text = train_df['full_text'].iloc[0]
print(f"ИСХОДНЫЙ ТЕКСТ: '{sample_text[:200]}...'")
print(f"ОБРАБОТАННЫЙ ТЕКСТ: '{preprocess_text(sample_text)}'")

# Применяем ко всем данным
tqdm.pandas(desc="Preprocessing Train")
train_df['processed_text'] = train_df['full_text'].progress_apply(preprocess_text)
tqdm.pandas(desc="Preprocessing Val")
val_df['processed_text'] = val_df['full_text'].progress_apply(preprocess_text)

ИСХОДНЫЙ ТЕКСТ: 'QA Engineer | Москва (м. Сколково / м. Крылатское) | getmatch agency | Python, Java, Go, C# | Наша компания сохраняет атмосферу стартапа, обеспечивая динамичность и стремление к результатам в сочетани...'
ОБРАБОТАННЫЙ ТЕКСТ: 'qa engineer москва м сколково м крылатский getmatch agency python java go c сохранять атмосфера стартап обеспечивать динамичность стремление результат сочетание комфорт стабильность корпоративный среда трудиться создание будущее российский мобильный связь ключевой продукт российский оператор базовый станция lte g разработать основа принцип открытый архитектура open ran приглашать талантливый специалист присоединиться просто сотрудник сторонник разделять страсть прогресс инновация предстоять заниматься участвовать процесс контроль качество разный этап жизненный цикл разрабатывать средство автоматизация функциональный нагрузочный тестирование разрабатывать тестовый документация участвовать лабораторный приёмочный испытание также составлять отчётный 

Preprocessing Train:   0%|          | 0/12500 [00:00<?, ?it/s]

Preprocessing Val:   0%|          | 0/4167 [00:00<?, ?it/s]

In [9]:
tqdm.pandas(desc="Preprocessing Test")
test_df['processed_text'] = test_df['full_text'].progress_apply(preprocess_text)

Preprocessing Test:   0%|          | 0/5556 [00:00<?, ?it/s]

In [ ]:
nltk.download('punkt')

In [ ]:
#создаем словарь
def build_vocab(texts, min_freq=10):
    counter = Counter()
    for text in texts:
        counter.update(text.split())
    vocab = {'<pad>':0, '<unk>':1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

def text2indices(text, word2ind):
    return [word2ind.get(w, word2ind['<unk>']) for w in text.split()]


In [ ]:
def collate_fn_with_padding(batch, pad_idx=0):
  #настройка объединения батчей в DataLoader
    texts, targets = zip(*batch)
    lengths = [len(t) for t in texts]
    max_len = max(lengths)

    padded_texts = []
    for t in texts:
        padded = t + [pad_idx] * (max_len - len(t))
        padded_texts.append(padded)
    padded_texts = torch.tensor(padded_texts, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.float)
    return padded_texts, targets

In [ ]:
class SalaryDataset(Dataset):
    def __init__(self, texts, targets, word2ind):
        self.texts = [text2indices(t, word2ind) for t in texts]
        self.targets = targets

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.targets[idx]

In [ ]:
vocab = build_vocab(train_df['processed_text'].tolist())

In [ ]:
print("Всего слов в словаре:", len(vocab))

In [ ]:
def train_model(model, train_dataloader, val_dataloader, num_epochs, device, vocab):
    # Критерий для регрессии - MSELoss, R2-score отслеживаем только на валидации
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())

    train_losses = []
    val_losses = []
    val_r2_scores = []

    for epoch in range(num_epochs):

        model.train()
        epoch_train_losses = []

        for x_batch, y_batch in tqdm(train_dataloader, desc=f'Training epoch {epoch+1}/{num_epochs}'):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad() # обнуляем градиенты
            preds = model(x_batch) # прямой проход
            loss = criterion(preds, y_batch) # лоссы

            loss.backward() # backward подсчет градиентов
            optimizer.step() # обновляем веса

            epoch_train_losses.append(loss.item())

        avg_train_loss = sum(epoch_train_losses) / len(epoch_train_losses)
        train_losses.append(avg_train_loss)


        model.eval()
        epoch_val_losses = []
        all_val_preds = []
        all_val_targets = []

        with torch.no_grad():
            for x_batch, y_batch in tqdm(val_dataloader, desc=f'Validation epoch {epoch+1}/{num_epochs}'):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)

                preds = model(x_batch)
                loss = criterion(preds, y_batch)

                epoch_val_losses.append(loss.item())

                # предсказания и истинные значения для R2
                all_val_preds.extend(preds.cpu().numpy())
                all_val_targets.extend(y_batch.cpu().numpy())

        avg_val_loss = sum(epoch_val_losses) / len(epoch_val_losses)
        val_losses.append(avg_val_loss)

        # R2- на всей валидационной выборке
        val_r2 = r2_score(all_val_targets, all_val_preds)
        val_r2_scores.append(val_r2)

        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train Loss = {avg_train_loss:.4f}, "
              f"Val Loss = {avg_val_loss:.4f}, "
              f"Val R2-score = {val_r2:.4f}")

    return train_losses, val_losses, val_r2_scores

In [ ]:
class SimpleRegressionModel(nn.Module):
  #простая модель регрессии, бейслайн
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128, pad_idx=0):
        super(SimpleRegressionModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        emb = self.embedding(x)  # (batch, seq_len, embed_dim)
        mask = (x != self.embedding.padding_idx).unsqueeze(-1)
        emb = emb * mask
        summed = emb.sum(dim=1)
        counts = mask.sum(dim=1).clamp(min=1e-9)
        avg_emb = summed / counts
        h = self.relu(self.fc1(avg_emb))
        out = self.fc2(h).squeeze(1)
        return out

In [ ]:
train_dataset = SalaryDataset(train_df['processed_text'].tolist(), train_df['log_salary_from'].tolist(), vocab)
val_dataset = SalaryDataset(val_df['processed_text'].tolist(), val_df['log_salary_from'].tolist(), vocab)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,
                          collate_fn=lambda batch: collate_fn_with_padding(batch, pad_idx=vocab['<pad>']))

#на валидации не перемешиваем
val_loader = DataLoader(val_dataset, batch_size=16,
                        collate_fn=lambda batch: collate_fn_with_padding(batch, pad_idx=vocab['<pad>']))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
simple_model = SimpleRegressionModel(len(vocab), pad_idx=vocab['<pad>']).to(device)

num_epochs = 10
train_losses, val_losses, val_r2_scores = train_model(
    simple_model, train_loader, val_loader, num_epochs, device, vocab
)
print("\nTraining completed!")
print(f"Final Train Loss: {train_losses[-1]:.4f}")
print(f"Final Val Loss: {val_losses[-1]:.4f}")
print(f"Final Val R2-score: {val_r2_scores[-1]:.4f}")

In [ ]:
plt.plot(np.arange(len(train_losses)), train_losses)
plt.plot(np.arange(len(val_losses)), val_losses)
plt.title('Losses')
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(np.arange(len(val_r2_scores)), val_r2_scores)
plt.title('val_r2_scores')
plt.xlabel("epoch")
plt.show()

In [ ]:
# dataset для теста
class TestSalaryDataset(Dataset):
    def __init__(self, texts, word2ind):
        self.texts = texts
        self.vocab = word2ind
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        tokens = self.texts[idx].split()
        indices = [self.vocab.get(t, self.vocab['<unk>']) for t in tokens]
        return indices

test_dataset = TestSalaryDataset(test_df['processed_text'].tolist(), vocab)

test_loader = DataLoader(test_dataset, batch_size=16,
                         collate_fn=lambda batch: collate_fn_with_padding([(x, 0) for x in batch], pad_idx=vocab['<pad>']))


simple_model.eval()
predictions = []
with torch.no_grad():
    for x_batch, _ in test_loader:
        x_batch = x_batch.to(device)
        preds = simple_model(x_batch)
        predictions.extend(preds.cpu().numpy())

In [21]:
submition = pd.read_csv(teta_nn_1_2025_path + '/sample_submition.csv')

In [ ]:
submition.prediction = predictions

In [ ]:
submition.to_csv('linear_sub.csv', index = False)

## Эксперимент с регрессионной моделью на GRU приводит к нестабильному результату:

In [ ]:
class GRURegressionModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim: int, num_layers: int, dropout_rate: float, pad_idx: int):
      #
        super().__init__()
        # Сохраняем для инициализации скрытого состояния:
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)

        #дополнительный линейный слой и тангенциальная активация
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=dropout_rate)

        self.output_projection = nn.Linear(hidden_dim, 1)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch)
        output, hidden = self.gru(embeddings)

        final_hidden_state = hidden[-1, :, :]  # последнее скрытое состояние

        processed_hidden = self.dropout(self.linear(self.non_lin(final_hidden_state)))

        prediction = self.output_projection(self.non_lin(processed_hidden)).squeeze(1)  # Регрессия

        return prediction


In [ ]:
model = GRURegressionModel(len(vocab), hidden_dim=256, num_layers=4, dropout_rate=0.2, pad_idx=vocab['<pad>']).to(device)

train_losses, val_losses, val_r2_scores = train_model(model, train_loader, val_loader, num_epochs=30, device=device, vocab=vocab)

print("\nTraining completed!")
print(f"Final Train Loss: {train_losses[-1]:.4f}")
print(f"Final Val Loss: {val_losses[-1]:.4f}")
print(f"Final Val R2-score: {val_r2_scores[-1]:.4f}")

In [ ]:
plt.plot(np.arange(len(train_losses)), train_losses)
plt.plot(np.arange(len(val_losses)), val_losses)
plt.title('Losses')
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(np.arange(len(val_r2_scores)), val_r2_scores)
plt.title('val_r2_scores')
plt.xlabel("epoch")
plt.show()

In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for x_batch, _ in test_loader:
        x_batch = x_batch.to(device)
        preds = model(x_batch)
        predictions.extend(preds.cpu().numpy())

In [ ]:
submition.prediction = predictions
submition.to_csv('GRU_sub.csv', index = False)

## DeepPavlov

In [36]:
model_name = "DeepPavlov/rubert-base-cased"

dp_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1  # регрессия
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
class BertRegressionDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=128):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.targets[idx], dtype=torch.float)
        return item


In [38]:
train_dataset = BertRegressionDataset(train_df['processed_text'].tolist(), train_df['log_salary_from'].tolist(), tokenizer)
val_dataset = BertRegressionDataset(val_df['processed_text'].tolist(), val_df['log_salary_from'].tolist(), tokenizer)

In [39]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [40]:
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import r2_score

def train_epoch(model, data_loader, optimizer, device):
    model.train()
    losses = []
    for batch in tqdm(data_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).unsqueeze(1)  # [batch, 1]

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = outputs.logits
        loss_fct = nn.MSELoss()
        loss = loss_fct(preds, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return sum(losses)/len(losses)

def eval_epoch(model, data_loader, device):
    model.eval()
    losses = []
    all_preds = []
    all_labels = []
    loss_fct = nn.MSELoss()
    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).unsqueeze(1)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = outputs.logits
            loss = loss_fct(preds, labels)
            losses.append(loss.item())
            all_preds.extend(preds.cpu().squeeze(1).tolist())
            all_labels.extend(labels.cpu().squeeze(1).tolist())
    avg_loss = sum(losses)/len(losses)
    r2 = r2_score(all_labels, all_preds)
    return avg_loss, r2


In [41]:
optimizer = optim.AdamW(dp_model.parameters(), lr=2e-5)
num_epochs = 7
for epoch in range(num_epochs):
    train_loss = train_epoch(dp_model, train_loader, optimizer, device)
    val_loss, val_r2 = eval_epoch(dp_model, val_loader, device)
    print(f'Epoch {epoch+1}: Train Loss {train_loss:.4f}, Val Loss {val_loss:.4f}, Val R2 {val_r2:.4f}')


100%|██████████| 131/131 [00:29<00:00,  4.50it/s]


Epoch 1: Train Loss 0.6050, Val Loss 0.1496, Val R2 0.6177


100%|██████████| 131/131 [00:28<00:00,  4.56it/s]


Epoch 2: Train Loss 0.1495, Val Loss 0.1333, Val R2 0.6596


100%|██████████| 131/131 [00:29<00:00,  4.49it/s]


...

Epoch 5: Train Loss 0.0739, Val Loss 0.1368, Val R2 0.6499


100%|██████████| 131/131 [00:28<00:00,  4.53it/s]


Epoch 6: Train Loss 0.0592, Val Loss 0.1488, Val R2 0.6195


100%|██████████| 131/131 [00:28<00:00,  4.52it/s]

Epoch 7: Train Loss 0.0468, Val Loss 0.1389, Val R2 0.6444


In [42]:
test_dataset = BertRegressionDataset(test_df['processed_text'].tolist(), [0]*len(test_df), tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)

In [43]:
dp_model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = dp_model(input_ids, attention_mask=attention_mask)
        preds = outputs.logits.squeeze(1).cpu().tolist()
        predictions.extend(preds)

In [26]:
submition.prediction = predictions
submition.to_csv('DeepPav_sub.csv', index = False)

## RuModernBert

In [27]:
model_name = "deepvk/RuModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bert_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1  # для регрессии
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at deepvk/RuModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [28]:
optimizer = optim.AdamW(bert_model.parameters(), lr=2e-5)
num_epochs = 7
for epoch in range(num_epochs):
    train_loss = train_epoch(bert_model, train_loader, optimizer, device)
    val_loss, val_r2 = eval_epoch(bert_model, val_loader, device)
    print(f'Epoch {epoch+1}: Train Loss {train_loss:.4f}, Val Loss {val_loss:.4f}, Val R2 {val_r2:.4f}')

100%|██████████| 261/261 [00:43<00:00,  5.95it/s]


Epoch 1: Train Loss 0.2217, Val Loss 0.1499, Val R2 0.6175


100%|██████████| 261/261 [00:43<00:00,  5.93it/s]


Epoch 2: Train Loss 0.1319, Val Loss 0.1374, Val R2 0.6492


100%|██████████| 261/261 [00:44<00:00,  5.92it/s]


...

Epoch 5: Train Loss 0.0399, Val Loss 0.1276, Val R2 0.6747


100%|██████████| 261/261 [00:44<00:00,  5.90it/s]


Epoch 6: Train Loss 0.0285, Val Loss 0.1254, Val R2 0.6802


100%|██████████| 261/261 [00:43<00:00,  5.93it/s]

Epoch 7: Train Loss 0.0164, Val Loss 0.1324, Val R2 0.6622


In [29]:
bert_model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        preds = outputs.logits.squeeze(1).cpu().tolist()
        predictions.extend(preds)

In [30]:
submition.prediction = predictions
submition.to_csv('uModernBERT.csv', index = False)

## Ансамбль

In [ ]:
class SimpleBertEnsemble:
    def __init__(self, simple_model, bert_model, bert_tokenizer, vocab, device, alpha=0.5, max_len=256):
        self.simple_model = simple_model.to(device)
        self.bert_model = bert_model.to(device)
        self.bert_tokenizer = bert_tokenizer
        self.vocab = vocab
        self.device = device
        self.alpha = alpha
        self.max_len = max_len

    def simple_texts_to_tensor(self, texts):
        texts_indices = [text2indices(t, self.vocab) for t in texts]
        max_len = max(len(x) for x in texts_indices)
        padded = [x + [self.vocab['<pad>']] * (max_len - len(x)) for x in texts_indices]
        return torch.tensor(padded, dtype=torch.long)

    def predict(self, texts, batch_size=16):
        self.simple_model.eval()
        self.bert_model.eval()

        preds_simple = []
        preds_bert = []

        # Предсказания SimpleRegressionModel
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            batch_tensor = self.simple_texts_to_tensor(batch_texts).to(self.device)
            with torch.no_grad():
                batch_preds = self.simple_model(batch_tensor)
                preds_simple.extend(batch_preds.cpu().tolist())

        # Предсказания BERT
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            encodings = self.bert_tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=self.max_len,
                return_tensors='pt'
            )
            input_ids = encodings['input_ids'].to(self.device)
            attention_mask = encodings['attention_mask'].to(self.device)

            with torch.no_grad():
                outputs = self.bert_model(input_ids, attention_mask=attention_mask)
                batch_preds = outputs.logits.squeeze(1)
                preds_bert.extend(batch_preds.cpu().tolist())

        # Усреднение с весами alpha
        preds_final = [
            self.alpha * s + (1 - self.alpha) * b
            for s, b in zip(preds_simple, preds_bert)
        ]
        return preds_final